In [1]:
import pandas as pd
import requests
import time
import re
import os

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service

dataIn = './../dataIn/'
dataOut = './../dataOut/'

In [2]:
# 크롬 드라이버 다운로드 :  https://googlechromelabs.github.io/chrome-for-testing/
chrome_options = webdriver.ChromeOptions() # 크롬 브라우저 옵션
drive_path = 'chromedriver.exe' # 다운로드 받은 드라이버 파일
myservice = Service(drive_path) # 드라이버 제어를 위한 서비스 객체
driver = webdriver.Chrome(service=myservice, options=chrome_options) # 드라이버 객체
print(type(driver)) # 객체가 잘 생성되었는 지 확인

wait_time = 10 # 최대 대기 시간 10초
driver.implicitly_wait(wait_time)

<class 'selenium.webdriver.chrome.webdriver.WebDriver'>


In [3]:
# driver.maximize_window()

In [4]:
# 스타 벅스 음료 목록 페이지
starbucks_beverage_url = 'https://www.starbucks.co.kr/menu/drink_list.do'
driver.get(starbucks_beverage_url)

In [5]:
# 스타벅스 음료 페이지의 html 코드를 파싱해서 html 파일에 기록합니다.
html = driver.page_source # 해당 페이지 소스 코드 반환
filename = dataOut + 'starbucks_beverage.html'
htmlFile = open(filename, mode='wt', encoding='UTF-8')
print(html, file=htmlFile)
htmlFile.close()
print(filename, '파일 생성됨')

./../dataOut/starbucks_beverage.html 파일 생성됨


In [6]:
image_info = [] # (출처, alt속성, 상품코드) 형식의 tuple을 담고 있는 list

try:
    # 음료 메뉴 이미지 로딩 대기하기
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'img')))

    time.sleep(3)

    # 모든 img 태그 추출
    img_elements = driver.find_elements(By.TAG_NAME, 'img')
    print(f'이미지 개수 : {len(img_elements)}')

    for img in img_elements:
        src = img.get_attribute('src')

        # 특정한 경로('skuimg'단어가 포함된)로 시작하는 이미지만 필터링
        if src and 'skuimg' in src: # None이 아닌 문자열 중에서 'skuimg'단어가 포함된...
            alt = img.get_attribute('alt')

            # 상품 코드 추출 : 정규 표현식으로 [ ] 사이의 숫자 추출
            # r은 raw string
            # (.*?) : 공백 문자를 제외한 모든 글자의 최소 매칭
            match = re.search(r'\[(.*?)\]', src)

            # 조건 표현식 : 자바의 삼항 연산자와 유사
            product_code = match.group(1) if match else None

            image_info.append((src, alt, product_code))
        # end if
    # end for
except Exception as err:
    print(err)
finally:
    pass # driver.quit()

이미지 개수 : 334


In [7]:
# 데이터 프레임으로 변환
df = pd.DataFrame(image_info, columns=['이미지Url', '상품명', '상품코드'])
print(f'총 {len(df)}개')

# csv 파일로 저장
csv_filename = dataOut + 'starbucks_beverage_images.csv'
df.to_csv(csv_filename, index=False, encoding='utf-8-sig')

총 205개


In [8]:
print('# 상위 몇 개만 보기')
print(df.head())

# 상위 몇 개만 보기
                                              이미지Url          상품명  \
0  https://image.istarbucks.co.kr/upload/store/sk...  나이트로 바닐라 크림   
1  https://image.istarbucks.co.kr/upload/store/sk...   나이트로 콜드 브루   
2  https://image.istarbucks.co.kr/upload/store/sk...     돌체 콜드 브루   
3  https://image.istarbucks.co.kr/upload/store/sk...     리저브 나이트로   
4  https://image.istarbucks.co.kr/upload/store/sk...    리저브 콜드 브루   

            상품코드  
0  9200000002487  
1  9200000000479  
2  9200000002081  
3  9200000002407  
4  9200000002093  


In [9]:
# 이미지를 저장할 폴더 생성
save_image_folder = 'drink_images'
os.makedirs(save_image_folder, exist_ok=True) # 이미 존재한다면 무시함

# 스타벅스 각 음료 세부 정보 url
starbucks_beverage_detail_url = 'https://www.starbucks.co.kr/menu/drink_view.do?product_cd='

# 변수 image_info 리스트를 반복하면서 상세 URL 생성
detail_urls = []

In [10]:
for img in image_info:
    img_url = img[0] # 이미지의 url 주소
    img_name = img[1] # 음료 이름

    if img_url and img_name:
        # 운영 체제에서 파일 이름으로 사용하지 못하는 글자에 유의
        safe_name = ''.join(name for name in img_name if name.isalnum() or name in ' _-')
        file_path = os.path.join(save_image_folder, f'{safe_name}.jpg')

        try: # HTTP Get 방식 요청
            response = requests.get(img_url)
            if response.status_code == 200: # 정상적으로 응답 받음
                # 바이트 이미지를 파일로 저장하기
                with open(file_path, 'wb') as f: # 'wb'는 하드 디스크에 바이너리로 다운로드함
                    f.write(response.content)
                # end with
                # print(f'저장 완료 : {file_path}')

            else:
                print(f'다운로드 실패 : {response.status_code}, URL : {img_url}')
            # end if

        except Exception as err:
            print(f'예외 발생 : {err}, URL : {img_url}')
        # end try

        product_code = img[2] # 상품 번호
        if product_code:
            detail_url = starbucks_beverage_detail_url + product_code
            detail_urls.append(detail_url)
        # end if
    # end if
# end for

print('작업이 완료 되었습니다.')

작업이 완료 되었습니다.


In [11]:
print('# 상위 몇개만 결과 출력하기')
for url in detail_urls[:5]:
    print(url)
# end for

# 상위 몇개만 결과 출력하기
https://www.starbucks.co.kr/menu/drink_view.do?product_cd=9200000002487
https://www.starbucks.co.kr/menu/drink_view.do?product_cd=9200000000479
https://www.starbucks.co.kr/menu/drink_view.do?product_cd=9200000002081
https://www.starbucks.co.kr/menu/drink_view.do?product_cd=9200000002407
https://www.starbucks.co.kr/menu/drink_view.do?product_cd=9200000002093


In [12]:
for url in detail_urls[:1]: # 첫번째 페이지에 대하여
    driver.get(url)
    html = driver.page_source # 해당 페이지 소스 코드 반환
    filename = dataOut + 'first_detail_page.html'
    htmlFile = open(filename, mode='wt', encoding='UTF-8')
    print(html, file=htmlFile)
    htmlFile.close()
    print(filename, '파일 생성됨')
# end for

./../dataOut/first_detail_page.html 파일 생성됨


In [27]:
all_table_data = []

In [28]:
table_elements= driver.find_elements(By.TAG_NAME, 'table')
print(f'개수 : {len(table_elements)}')

개수 : 10
